<a href="https://colab.research.google.com/github/nthrsn/transit-safety-insights/blob/nthrsn-patch-1/log_reg_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#import the necessary libraries and load the dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
# Create a directory for the data
!mkdir -p data

# Download the dataset from data.transportation.gov
!wget -O data/raw_data.csv "https://data.transportation.gov/api/views/9ivb-8ae9/rows.csv?accessType=DOWNLOAD"

--2025-03-19 00:34:22--  https://data.transportation.gov/api/views/9ivb-8ae9/rows.csv?accessType=DOWNLOAD
Resolving data.transportation.gov (data.transportation.gov)... 52.206.68.26, 52.206.140.205, 52.206.140.199
Connecting to data.transportation.gov (data.transportation.gov)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘data/raw_data.csv’

data/raw_data.csv       [    <=>             ]  91.30M  4.15MB/s    in 21s     

2025-03-19 00:34:44 (4.33 MB/s) - ‘data/raw_data.csv’ saved [95738829]



In [16]:
# Enable inline plotting for Jupyter Notebook
%matplotlib inline

# Set a style for the plots
sns.set(style="whitegrid")

# Load the dataset
data_path = "data/raw_data.csv"
df = pd.read_csv(data_path)

# Display the first few rows of the dataframe
df.head()

<ipython-input-16-2b7f19373cd1>:9: DtypeWarning: Columns (18,20,24,34,38,45,46,49,53,54,60,63,64,69,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


,NTD ID,Agency,Primary UZA UACE Code,Rail/Bus/Ferry,Mode Name,Mode,TOS,Fixed Route Flag,Year,Event Description,...,Occupant of Other Vehicle Injuries,Occupant of Other Vehicle Serious Injuries,Other Injuries,Other Serious Injuries,Suicide Injuries,Suicide Serious Injuries,Trespasser Inuries (Subtotal),Trespasser Serious Injuries (Subtotal),Total Serious Injuries,Person List
0,20080,New Jersey Transit Corporation,63217,Bus,Bus,MB,DO,True,2024,Bus was parked in <REDACTED> <REDACTED> and an...,...,0,0,0,0,0,0,0,0,0,NaN
1,40035,Central Florida Regional Transportation Authority,65863,Bus,Demand Response,DR,DO,False,2024,Another vehicle made other front contact with ...,...,1,0,0,0,0,0,0,0,0,I Operator Inside Adult (19 to 60) F ; I ...
2,50027,Metro Transit,57628,Rail,Light Rail,LR,DO,True,2024,Officers responded to a shooting . Suspect was...,...,0,0,0,0,0,0,0,0,0,F Passenger Adult (19 to 60) M
3,90026,San Diego Metropolitan Transit System,78661,Rail,Light Rail,LR,DO,True,2024,San Diego Police took a female into custody af...,...,0,0,0,0,0,0,0,0,0,I Wait Leave Adult (19 to 60) M Y
4,20080,New Jersey Transit Corporation,63217,Bus,Bus,MB,DO,True,2024,operator was driving down <REDACTED> <REDACTED...,...,1,0,0,0,0,0,0,0,0,I Occ Other Veh Adult (19 to 60) M


In [17]:
# Get a list of all column names
column_names = df.columns.tolist()

# Print the list of column names
column_names

['NTD ID',
 'Agency',
 'Primary UZA UACE Code',
 'Rail/Bus/Ferry',
 'Mode Name',
 'Mode',
 'TOS',
 'Fixed Route Flag',
 'Year',
 'Event Description',
 'Incident Number',
 'Event Date',
 'Event Time',
 'Event Type',
 'Event Type Group',
 'Transit Worker Type',
 'Transit Worker Assault Detail Type',
 'Transit Worker Assault Flag',
 'Other Event Type Description',
 'Event Category',
 'Runaway Train Flag',
 'Safety/Security',
 'Collision With',
 'Property Damage',
 'Substantial Property Damage Type',
 'Total Injuries',
 'Total Fatalities',
 'Towed (Y/N)',
 'Number of Transit Vehicles Involved',
 'Number of Non-Transit Vehicles Involved',
 'Number of Cars on Involved Transit Vehicles',
 'Number of Derailed Cars on Involved Transit Vehicles',
 'Non-Transit Vehicle Type List',
 'Location Type',
 'Multiple Locations List',
 'Approximate Address',
 'Latitude',
 'Longitude',
 'LatLon',
 'Evacuation',
 'Evacuation for Life Safety (Y/N)',
 'Evacuation to Right-of-Way',
 'Evacuation Comment',
 'Sel

In [18]:
# Define the list of columns to keep
columns_to_keep = [
    'NTD ID',
    'Primary UZA UACE Code',
    'Rail/Bus/Ferry',
    'Mode Name',
    'Mode',
    'TOS',
    'Year',
    'Event Date',
    'Event Time',
    'Event Type',
    'Event Type Group',
    'Safety/Security',
    'Property Damage',
    'Total Injuries',
    'Total Fatalities',
    'Towed (Y/N)',
    'Number of Transit Vehicles Involved',
    'Number of Non-Transit Vehicles Involved',
    'Number of Cars on Involved Transit Vehicles',
    'Non-Transit Vehicle Type List',
    'Location Type',
    'Latitude',
    'Longitude',
    'Weather',
    'Lighting',
    'Road Configuration',
    'Path Condition',
    'Right of Way Condition',
    'Intersection Control Device',
    'Transit Vehicle Action',
    'Other Transit Vehicle Action Description',
    'Non-Transit Vehicle Action List',
    'Transit (Y/N)',
    'Fuel Type',
    'Vehicle Speed',
    'Transit Vehicle Type',
    'Non-Transit Vehicle Type',
    'Transit Vehicle Manufacturer',
    'Total Serious Injuries'
]

df = df[columns_to_keep]

In [19]:
# count the number of missing values in each column
missing_values = df.isnull().sum()

# drop columns with more than 10% missing values
columns_to_drop = missing_values[missing_values > 0.1 * df.shape[0]].index
df = df.drop(columns=columns_to_drop)

# drop rows with missing values
df = df.dropna()

# drop duplicates
df = df.drop_duplicates()

# save the cleaned data, save to data folder as data_cleaned.csv
# df.to_csv('../data/cleaned_data.csv', index=False)

# print the number of rows and columns in the cleaned data
print(df.shape)

# print the number of missing values in the cleaned data
print(df.isnull().sum())

# print the number of duplicates in the cleaned data
print(df.duplicated().sum())

# print the first few rows of the cleaned data
print(df.head())

(98611, 20)
NTD ID                                         0
Primary UZA UACE Code                          0
Rail/Bus/Ferry                                 0
Mode Name                                      0
Mode                                           0
TOS                                            0
Year                                           0
Event Date                                     0
Event Time                                     0
Event Type                                     0
Event Type Group                               0
Safety/Security                                0
Total Injuries                                 0
Total Fatalities                               0
Towed (Y/N)                                    0
Number of Transit Vehicles Involved            0
Number of Non-Transit Vehicles Involved        0
Number of Cars on Involved Transit Vehicles    0
Location Type                                  0
Total Serious Injuries                         0
dtype: i

In [20]:
import pandas as pd
import numpy as np

# For modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [21]:
# 3. Filter to rows where Safety/Security = "SFT"
df = df[df["Safety/Security"] == "SFT"]

# 4. Define the binary target (Major_Incident)
#    Criteria: (Total Serious Injuries > 0) OR (Total Fatalities > 0) OR (Property Damage > 100,000)
DAMAGE_THRESHOLD = 100_000

df["Major_Incident"] = df.apply(
    lambda row: 1 if (
        row.get("Total Serious Injuries", 0) > 0
        or row.get("Total Fatalities", 0) > 0
        or row.get("Property Damage", 0) > DAMAGE_THRESHOLD
    ) else 0,
    axis=1
)

In [22]:
# 5. Create features (X) and target (y)
y = df["Major_Incident"]
X = df.drop(["Major_Incident"], axis=1)

# 6. Basic data cleaning / preprocessing

# Identify numeric vs. object (categorical) columns
numeric_cols = X.select_dtypes(include=[np.number]).columns
categorical_cols = X.select_dtypes(include=["object"]).columns

# Fill missing numeric values with 0 (simple approach)
X[numeric_cols] = X[numeric_cols].fillna(0)

# Factorize the categorical columns
for col in categorical_cols:
    X[col] = X[col].fillna("MissingCategory")  # handle NaN in categorical columns
    X[col] = X[col].astype(str).factorize()[0]

# 7. Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 8. Train a simple logistic regression model
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)

# 9. Make predictions
y_pred = logreg.predict(X_test)

# 10. Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.992782275779828

Confusion Matrix:
 [[15309    32]
 [   83   509]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     15341
           1       0.94      0.86      0.90       592

    accuracy                           0.99     15933
   macro avg       0.97      0.93      0.95     15933
weighted avg       0.99      0.99      0.99     15933



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
